### ***Centroid based Text Summarization***

---

### Modules ###

In [26]:
import numpy as np
import helper as h

import operator
from functools import reduce

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

### Example text ###

In [27]:
text = '''
Głównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz. Akcja toczy się w roku 1602 roku gdzie wraz z resztą braci bierze udział w polowaniu na czarownice. Jednak niespodziewane spotkanie swojej przyrodniej siostry Anny Held, osądzanej o wykonywanie czarów, powoduje że Laymann odrzuca regułę klasztoru. Podstępnie zakrada się do biblioteki gdzie studiuje zakazane księgi. Zostaje przyłapany przez swojego nauczyciela brata Jakoba i uwięziony w lochu.
Główny bohater spotyka tam starszego człowieka Lugaida Vandroiy, który przedstawia mu się jako druid (Reach Out For The Light). Spotkany człowiek opowiada Gabrielowi o innym zagrożonym wymiarze – Avantasii. W zamian za pomoc w uratowaniu Avantasii druid obiecuje uratowanie Anny. Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy. Vandroiy używa go by przenieść Gabriela do równoległego świata.
W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome). Niosą ze sobą także księgę odkrytą przez Gabriela. Wedle starożytnego zapisu wynika że księga jest ostatnią siódmą częścią pieczęci, która w całości daje właścicielowi absolutną wiedzę gdy tylko dostanie się on do wieży wyznaczającej środek Avantasii.
Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside). Opowiadają mu o toczącej się wojnie przeciwko siłom zła oraz o planach papieża (Sign Of The Cross). Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy. Gabriel przybywa w momencie gdy Clemens VIII rozmawia z tajemniczym głosem dochodzącym z wieży. Zręcznemu Gabrielowi udaje się skraść pieczęć papieżowi po czym zanosi ją do miasta elfów (The Tower). Zdarzenie to kończy pierwszy album.
Jednak Gabriel nie jest usatysfakcjonowany. Chce dowiedzieć się więcej o świecie Avantasii dlatego Elderane wysyła nowicjusza do drzewa poznania. Tam Gabriel podczas objawienia widzi brata Jakoba który znosił okropny ból w jeziorze ognia (The Final Sacrifice). Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach. Kielich jest więzieniem dla ogromnej ilości torturowanych dusz, artefakt strzeżony jest także przez siejącą postrach bestię. Mimo niepowodzeń delfickich wypraw, Gabriel i Regrin powracają na ziemię by zmierzyć się z bestią. Przyjaciele znajdują kielich i przewracają go co umożliwia ucieczkę duszom. Przebudzona bestia zabija jednak krasnoluda, Gabrielowi udaje się uciec.
Gabriel wraca do Vandroiya, który czekał na niego. Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę. Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić Annę. Falk von Kronberg nakrywa ich i każe aresztować. Rozpoczęła się walka w której poległ Vandroiy raniony przez Kronberga, który później zostaje uśmiercony przez brata Jakoba. Anna ucieka by ponownie złączyć się z Gabrielem. Podążają wspólnie nieznaną drogą w przyszłość (Into The Unknown).
'''

Source: https://pl.wikipedia.org/wiki/Avantasia

### Split text into sentences ###

Make a list of raw sentences as well as preprocessed sentences

In [28]:
sentences, raw_sentences = h.prepare_text(text)

In [29]:
sentences

['głównym bohaterem gabriel laymann nowicjusz klasztoru dominikanów mainz',
 'akcja toczy wraz resztą braci bierze udział polowaniu czarownice',
 'niespodziewane spotkanie swojej przyrodniej siostry anny held osądzanej wykonywanie czarów powoduje laymann odrzuca regułę klasztoru',
 'podstępnie zakrada biblioteki studiuje zakazane księgi',
 'zostaje przyłapany swojego nauczyciela brata jakoba uwięziony lochu',
 'główny bohater spotyka starszego człowieka lugaida vandroiy przedstawia druid reach out for the light',
 'spotkany człowiek opowiada gabrielowi innym zagrożonym wymiarze – avantasii',
 'zamian pomoc uratowaniu avantasii druid obiecuje uratowanie anny',
 'razem udaje uciec lochów breaking away czym vandroiy zabiera gabriela kamieniołomu ukryty portal łączący oba światy',
 'vandroiy używa przenieść gabriela równoległego świata',
 'czasie biskup mainz johann adam von bicken brat jakob rządca falk von kronberg drodze rzymu zamierzają spotkać papieżem clemensem glory of rome',
 'nios

### TF-IDF pipeline ###

Prepare TF-IDF pipeline for centroid building

In [30]:
tfidf = Pipeline([
    ('count', CountVectorizer()),
    ('tfidf', TfidfTransformer(norm = None, sublinear_tf = False, smooth_idf = False))
])

In the begging `tfidf.fit_transform(sentences).toarray()` produces a list of shape (n,m) where: 

n - number of sentences, 

m - number of unique tokens across all sentences (vocabulary length)

In [6]:
tfidf.fit_transform(sentences).toarray().shape

(34, 263)

In [7]:
tfidf.fit_transform(sentences).toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 4.52636052, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

### Make centroids ###

Sum up the numbers by column and divide it by the maximum number to obtaint values from 0 to 1 (probability)

In [8]:
centroid_vector_all = tfidf.fit_transform(sentences).toarray().sum(axis = 0)
centroid_vector_all = np.divide(centroid_vector_all, centroid_vector_all.max())

In [12]:
RELEVANT_VECTOR_CUTOFF_RATIO = 0.3 # cutt-of (threshold probability)

Return indices of tokens where the probability is above threshold

In [13]:
relevant_vector_indices = np.where(centroid_vector_all > RELEVANT_VECTOR_CUTOFF_RATIO)[0]

Make centroids (return tokens based on indices)

In [14]:
features = tfidf['count'].get_feature_names_out()
word_list = list(np.array(features)[relevant_vector_indices])

In [15]:
word_list # centroids

['anny',
 'annę',
 'avantasii',
 'brata',
 'czym',
 'druid',
 'elderane',
 'falk',
 'gabriel',
 'gabriela',
 'gabrielowi',
 'jakoba',
 'kielich',
 'klasztoru',
 'krasnoluda',
 'kronberg',
 'laymann',
 'mainz',
 'oba',
 'of',
 'opowiada',
 'pieczęci',
 'równoległego',
 'the',
 'uciec',
 'udaje',
 'uwolnić',
 'vandroiy',
 'von',
 'wieży',
 'zakrada',
 'zostaje',
 'świata',
 'światy']

### A lookup model ###

A lokkup model in this case is a Word2Vec dictionary with predefined size and window.

In [16]:
# list of all tokens across all sentences
all_words = [word_tokenize(sent) for sent in sentences]
all_words_flattened = reduce(operator.concat, all_words)

# standard (empty) Word2Vec model
model = Word2Vec(all_words, window=2, size=100, sg = 1, min_count=1)

# populate Word2Vec dictionary
model_lookup = dict()
for word in all_words_flattened:
    model_lookup[word] = model.wv[word]

### Centroids vector representation ###

In [17]:
def make_vector_representation(words: list, model_lookup: dict, model: Word2Vec) -> np.array:
    '''
    Make a vector representation of given list of words

    Args:
        words: list of words (for example all words across texts or list of centroids)
        model_lookup: Word2Vec dictionary
        model: Word2Vec predifined model
    
    Returns:
        Vector representation of given list of words
    '''

    # zeroes vector with length equal to model size (100) 
    representation = np.zeros(model.vector_size, dtype='float32')

    # get every word embedding (vector of size 100) and add it to the final
    # representation
    for word in words:
        if word in model_lookup.keys():
            representation += model_lookup[word]
    
    representation = np.divide(representation, len(words)) #normalization

    return representation

In [18]:
# 100 element vector representing all centroids
centroid_vector = make_vector_representation(word_list, model_lookup, model)

In [19]:
centroid_vector

array([-2.89565924e-05,  9.65558229e-06,  1.77947848e-04, -5.34630322e-04,
       -1.04064682e-04,  3.02239496e-04, -3.98625009e-04,  2.07094781e-04,
       -5.32309467e-04, -9.13270283e-04, -2.25071504e-04, -3.90627538e-04,
       -1.04044266e-04,  1.08902050e-05, -5.09153761e-05,  2.24534539e-04,
       -2.63228692e-04,  1.48714853e-05,  3.26290174e-04, -2.43790972e-04,
       -2.41780814e-04, -2.76955048e-04, -1.90241059e-04,  5.29986864e-04,
       -1.95764835e-04,  1.53886474e-04,  1.39786833e-04,  4.86147357e-04,
       -9.86402156e-04, -3.09635361e-04,  3.90545232e-04,  3.11998388e-04,
       -5.94873063e-06,  3.46208981e-04,  1.27840221e-05,  2.76540759e-05,
       -5.86205388e-05,  9.08914371e-05,  8.14602245e-04, -1.90259219e-04,
        5.42688183e-04, -7.52644246e-06,  1.89640967e-04,  5.86976879e-04,
        1.55038506e-05,  4.57328948e-04,  8.83935427e-04,  1.14127924e-03,
        2.37550965e-04, -4.07968968e-04, -3.58466437e-04, -8.95125631e-05,
       -7.69597595e-04,  

### Sentence similarity ###

Make vector respresentation of every sentence `(sentence_vector)` and calculate their similarity score 

In [20]:
sent_scores = dict() # empty dictionary for sentence representation storage
for n, sentence in enumerate(sentences):
    words = sentence.split()

    # vector representation of a particular sentence
    sentence_vector = make_vector_representation(words, model_lookup, model)

    # how similar a particular sentence is compared to centroids
    score = h.cos_sim(sentence_vector, centroid_vector)
    sent_scores[n] = [score, sentences[n], sentence_vector, raw_sentences[n]]

# sorted sentenced based on their similarity with the centroids
sent_scores_sort = sorted(sent_scores.items(), key = lambda item: item[1][0], reverse=True)

### Redundancy handling ###

In [21]:
# don't return sentences that are similar to each other more than
COS_SIM_CUT_OFF = 0.75

In [23]:
for s in sent_scores_sort: 
    count = 0
    sentences_summary = []
    for s in sent_scores_sort:
        # if sentences in final summary have more then 100 tokens break
        if count > 100: 
            break
        # if sentences in final summary have less then 100 tokens set flag
        include_flag = True

        # check sim score of every sentence (s) with every other sentence (ps)
        for ps in sentences_summary:
            sim = h.cos_sim(s[1][2], ps[1][2])

            # if similarity if above threshold don't include the sentence
            if sim > COS_SIM_CUT_OFF:
                include_flag = False

        # include only sentences with include_flag = True
        if include_flag:
            sentences_summary.append(s) # add sentence to final summary
            count += len(s[1][1].split()) # add sentence length to counter
    
    sentences_summary = sorted(sentences_summary, key=lambda el: el[0], reverse=False)

Extract only sentences from sentences_summary

In [24]:
summarized_sents = []
for n, sent in enumerate(sentences_summary):
    summarized_sents.append(sentences_summary[n][1][3])

summarized_sents

['\nGłównym bohaterem jest Gabriel Laymann, nowicjusz klasztoru dominikanów w Mainz',
 'Razem udaje im się uciec z lochów (Breaking Away) po czym Vandroiy zabiera Gabriela do kamieniołomu, gdzie ukryty jest portal łączący oba światy',
 'Vandroiy używa go by przenieść Gabriela do równoległego świata',
 'W tym czasie biskup Mainz Johann Adam von Bicken, brat Jakob oraz rządca Falk von Kronberg są w drodze do Rzymu gdzie zamierzają spotkać się z papieżem Clemensem VIII (Glory of Rome)',
 'Gdy Gabriel dostaje się do równoległego świata jest powitany przez dwóch jego mieszkańców elfa Elderana oraz krasnoluda Regrina (Inside)',
 'Jeżeli papież użyje pieczęci połączenie między Avantasią a światem ludzi zostanie zamknięte, a oba światy dotkną straszliwe kataklizmy',
 'Elderane opowiada Gabrielowi o złotym kielichu ukrytym w rzymskich katakumbach',
 'Druid spełnia obietnice, zakrada się do więzienia by uwolnić Annę',
 'Jednak znajduje tam „przemienionego” brata Jakoba który także chce uwolnić A

### Visualize final result ###

In [25]:
h.display_highlights(raw_sentences, summarized_sents)